In [ ]:
!pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite
!pip install -qU langchain-openai

In [2]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

··········


In [25]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in Malang")
print(search_results)
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Malang', 'region': 'East Java', 'country': 'Indonesia', 'lat': -8.0017, 'lon': 114.1367, 'tz_id': 'Asia/Jakarta', 'localtime_epoch': 1729058539, 'localtime': '2024-10-16 13:02'}, 'current': {'last_updated_epoch': 1729058400, 'last_updated': '2024-10-16 13:00', 'temp_c': 29.3, 'temp_f': 84.8, 'is_day': 1, 'condition': {'text': 'Partly Cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 200, 'wind_dir': 'SSW', 'pressure_mb': 1012.0, 'pressure_in': 29.88, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 34, 'cloud': 29, 'feelslike_c': 28.7, 'feelslike_f': 83.7, 'windchill_c': 29.3, 'windchill_f': 84.8, 'heatindex_c': 28.7, 'heatindex_f': 83.7, 'dewpoint_c': 12.1, 'dewpoint_f': 53.7, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 9.6, 'gust_mph': 6.7, 'gust_kph': 10.8}}"}, {'url': 'https://www.bbc.com/weather/1636722', 'content': '14-day weather 

In [26]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

··········


In [27]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [28]:
model_with_tools = model.bind_tools(tools)

In [29]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [30]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in Malang?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Malang'}, 'id': 'call_n6tJbdUBwEyJyBgQcf7tPz9e', 'type': 'tool_call'}]


In [31]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [32]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='bfe91124-3467-4ac6-bf96-27fa64e6b6f4'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-163d0eeb-6fba-4133-921b-ed2cbdbec1a3-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})]

In [12]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in malang?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in malang?', additional_kwargs={}, response_metadata={}, id='c8870ef8-11c9-4ee1-bc6f-5a30e299ea21'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_W9El2W5Vt8xfEPuIO12dWYfL', 'function': {'arguments': '{\n  "query": "current weather in Malang"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 89, 'total_tokens': 112, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4726b259-a517-4763-985a-2c146bd5d98e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Malang'}, 'id': 'call_W9El2W5Vt8xfEPuIO12dWYfL', 'type': 'tool_call'}], usage_metadata={'input_tokens': 89, 'output_

In [13]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in malang?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_FJL4hWz1pjx9nfA6y0WQ6ZoZ', 'function': {'arguments': '{\n  "query": "current weather in Malang"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 89, 'total_tokens': 112, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ee07cc15-6f48-4a62-a650-4cc39b09cdb4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Malang'}, 'id': 'call_FJL4hWz1pjx9nfA6y0WQ6ZoZ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 89, 'output_tokens': 23, 'total_tokens': 112, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})]

In [33]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in malang?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: tavily_search_results_json with inputs: {'query': 'current weather in Malang'}
Done tool: tavily_search_results_json
Tool output was: content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Malang\', \'region\': \'East Java\', \'country\': \'Indonesia\', \'lat\': -8.0017, \'lon\': 114.1367, \'tz_id\': \'Asia/Jakarta\', \'localtime_epoch\': 1729058576, \'localtime\': \'2024-10-16 13:02\'}, \'current\': {\'last_updated_epoch\': 1729058400, \'last_updated\': \'2024-10-16 13:00\', \'temp_c\': 29.3, \'temp_f\': 84.8, \'is_day\': 1, \'condition\': {\'text\': \'Partly Cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/116.png\', \'code\': 1003}, \'wind_mph\': 2.2, \'wind_kph\': 3.6, \'wind_degree\': 200, \'wind_dir\': \'SSW\', \'pressure_mb\': 1012.0, \'pressure_in\': 29.88, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 34, \'cloud\': 29, \'feelslike_c\': 28.7, \'feelslike_f\': 83.7, \'windchill_c\': 29.3, \'windchill_f\': 8

In [19]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [20]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [21]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 85, 'total_tokens': 96, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1ef922fa-5c75-427a-be93-24284c0c9aea-0', usage_metadata={'input_tokens': 85, 'output_tokens': 11, 'total_tokens': 96, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})]}}
----


In [22]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 108, 'total_tokens': 114, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5baac4ee-4dd3-43b2-8283-6028790b87ec-0', usage_metadata={'input_tokens': 108, 'output_tokens': 6, 'total_tokens': 114, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})]}}
----


In [23]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="As an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 86, 'total_tokens': 125, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8a127173-550e-4387-875a-0e116a0d4133-0', usage_metadata={'input_tokens': 86, 'output_tokens': 39, 'total_tokens': 125, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})]}}
----
